# 2022 Sydney University Honours project - Novel kernels for deep learning on light field images

In [1]:
import numpy as np
import os
import PIL
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import sys, glob, os, random
import pandas as pd
import scipy.io as sio
import preprocessing.flatten
import preprocessing.hci_dataset_tools.file_io as hci_io
print('tensorflow version: ', tf.__version__)
print('Num GPUs Available: ', len(tf.config.list_physical_devices('GPU')))

tensorflow version:  2.9.1
Num GPUs Available:  1


2022-07-15 18:53:28.518126: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-15 18:53:28.598482: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-15 18:53:28.598629: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


## Data Loading and Prepreprocessing

In [3]:
# check disparity and depth maps
data_path = '../../datasets'
hci_boxes = '/hci_dataset/training/boxes/'

disparity = hci_io.read_disparity(data_path + hci_boxes)
print('disparity shape: \n', disparity.shape)
depth = hci_io.read_depth(data_path + hci_boxes)
print('depth shape: \n', depth.shape)

disparity file read success:  ../../datasets/hci_dataset/training/boxes/gt_disp_lowres.pfm
disparity shape: 
 (512, 512)
depth file read success:  ../../datasets/hci_dataset/training/boxes/gt_depth_lowres.pfm
depth shape: 
 (512, 512)


In [4]:
#hci_boxes_stacked = '/hci_dataset/training/boxes/stacked/'

img_shape = (7,512,7,512,3)

def load_dataset(num_imgs=1, 
                 read_dirs=[data_path+hci_boxes]):
    '''
    load images and depth maps into tensorflow dataset (from HCI) 
    '''
    labels = []
    img_set = []
    for i in range(num_imgs):
        img = Image.open(read_dirs[i] + '/stacked/stacked.png')
        img = np.asarray(img)
        img = img.reshape(img_shape, order='F')
        img_set.append(img)
        # read depth map as labels
        depth = hci_io.read_depth(data_path + hci_boxes)
        labels.append(depth)
    img_set = np.asarray(img_set)
    labels = np.asarray(labels)
    print(img_set.shape)
    dataset = tf.data.Dataset.from_tensor_slices((img_set, labels))
    return dataset

In [ ]:
dataset = load_dataset()

In [9]:
for elem in dataset:
    print(elem[0].numpy().shape)

(7, 512, 7, 512, 3)


## Model init

In [10]:
model = keras.Sequential([
                    keras.layers.Conv2D(input_shape=(7,512,7,512,3), filters=4, 
                                        kernel_size=3, strides=2, activation='relu', 
                                        name='Conv1'),
                      keras.layers.Flatten(),
                      keras.layers.Dense(1, name='Dense')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 7, 512, 3, 255, 4  112       
                             )                                   
                                                                 
 flatten_1 (Flatten)         (None, 10967040)          0         
                                                                 
 Dense (Dense)               (None, 1)                 10967041  
                                                                 
Total params: 10,967,153
Trainable params: 10,967,153
Non-trainable params: 0
_________________________________________________________________


## Training

In [11]:
epochs = 5

model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])
model.fit(dataset, epochs=epochs)

#test_loss, test_acc = model.evaluate(test_images, test_labels)
#print('\nTest accuracy: {}'.format(test_acc))

Epoch 1/5


ValueError: in user code:

    File "/home/saze/anaconda3/envs/meme/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/saze/anaconda3/envs/meme/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/saze/anaconda3/envs/meme/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/saze/anaconda3/envs/meme/lib/python3.9/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/home/saze/anaconda3/envs/meme/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/saze/anaconda3/envs/meme/lib/python3.9/site-packages/keras/engine/input_spec.p